In [1]:
import re
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import numpy as np

In [2]:
tracked_icd10s = [
    "T560",
    "J628",
    "J610",
    "J920",
    "C450",
    "C451",
    "C452",
    "C457",
    "C459",
    "T600",
    "T601",
    "T602",
    "T603",
    "T604",
    "T608",
    "T609",
    "T560",
    "J441",
    "๋J450",
    "I450",
    "I240",
    "I220",
    "I223",
    "T561",
    "T562",
    "T563",
    "T564",
    "T565",
    "T566",
    "T567",
    "T568",
    "T569",
    "H833",
    "H903",
    "H904",
    "H905",
    "T520",
    "T521",
    "T522",
    "T523",
    "T524",
    "T528",
    "T529",
    "G560",
    "Y960",
    "T670",
    "T671",
    "T672",
    "T673",
    "T674",
    "T675",
    "T676",
    "T677",
    "T678",
    "T679",
    "J450",
    ]

In [3]:
username = "creep" 
password = "fvmgv"
address = "192.168.5.4"
port = "3306"
database_name = "hos"
url = f"mysql+mysqlconnector://{username}:{password}@{address}:{port}/{database_name}"
engine = sqlalchemy.create_engine(url)

In [4]:
def call_database(sql):
    with engine.connect().execution_options(autocommit=False) as conn:
        query = conn.execute(text(sql))
        df = pd.DataFrame(query.fetchall())
        return df

def finder(visit_date:str, icd10s:list[str]):
    joined_icd10 = ",".join([f"\"{icd10}\"" for icd10 in icd10s])
    main_query = f'''
    select  PA.pname,PA.fname,PA.lname,PA.birthday,PA.sex,OV.hn,OV.vn,OV.vstdate,OV.diagtype,OV.doctor,PT.name,OV.icd10,I1.name from hos.ovstdiag AS OV
    left join hos.patient as PA
    on PA.hn = OV.hn
    left join hos.vn_stat as VS
    on VS.vn = OV.vn
    left join hos.pttype as PT
    on VS.pttype = PT.pttype
    inner join hos.icd101 as I1
    on OV.icd10 = I1.code
    where icd10 in ({joined_icd10}) AND OV.vstdate >= "{visit_date}"
    '''
    return call_database(main_query)

In [5]:
data = finder("2022-02-09", tracked_icd10s)

In [58]:
from fhir.resources.fhirtypes import String, Id, HumanNameType, IdentifierType
from fhir.resources.identifier import Identifier
from fhir.resources.patient import Patient
from fhir.resources.encounter import Encounter

p = Patient.construct()
p.name = [HumanNameType(use="official", family="family",
                        given=["a"], text="a family")]
p.identifier = [IdentifierType(system="https://dopa.go.th", value="123456F")]
p.id = Id(f"cid-{1234}")
p.json()
e = Encounter.construct()
# p.name = []


'{"resourceType": "Patient", "id": "cid-1234", "identifier": [{"system": "https://dopa.go.th", "value": "123456F"}], "name": [{"use": "official", "text": "a family", "family": "family", "given": ["a"]}]}'